In [ ]:
import cv2
from ultralytics import YOLO
import random

# ==============================
# CONFIG
# ==============================
STREAM_URL = "http://172.18.230.94:8080/video"
MODEL_PATH = "best.pt"
CONF_THRES = 0.4

# ==============================
# GENERATE COLORS FOR EACH CLASS
# ==============================
def generate_colors(num_classes):
    """Generate distinct colors for each class"""
    random.seed(42)  # For consistency
    colors = []
    for i in range(num_classes):
        colors.append((
            random.randint(0, 255),
            random.randint(0, 255),
            random.randint(0, 255)
        ))
    return colors

# ==============================
# LOAD MODEL
# ==============================
model = YOLO(MODEL_PATH)
print("✅ YOLO model loaded:", model.names)

# Generate colors for each class
num_classes = len(model.names)
class_colors = generate_colors(num_classes)
print(f"✅ Generated {num_classes} colors for classes")

# ==============================
# VIDEO STREAM
# ==============================
cap = cv2.VideoCapture(STREAM_URL)
if not cap.isOpened():
    print("❌ Cannot open IP Webcam stream")
    exit()

print("✅ Stream opened")

# ==============================
# MAIN LOOP
# ==============================
while True:
    ret, frame = cap.read()
    if not ret:
        print("❌ Frame grab failed")
        continue
    
    # ---------- YOLO INFERENCE ----------
    results = model(frame, conf=CONF_THRES, verbose=False)
    
    for r in results:
        if r.boxes is None:
            continue
        
        for box in r.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            conf = float(box.conf[0])
            cls = int(box.cls[0])
            
            # Get class name and color
            class_name = model.names[cls]
            color = class_colors[cls] 
            
            # Bounding box
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            
            # Label
            label = f"{class_name} {conf:.2f}"
            cv2.putText(
                frame,
                label,
                (x1, y1 - 8),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.6,
                color,  
                2
            )
    
    # ------------------------------------
    cv2.imshow("IP Webcam + YOLO", frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

✅ YOLO model loaded: {0: 'imp_item', 1: 'imp_item_stolen', 2: 'person_1', 3: 'person_2'}
✅ Generated 4 colors for classes
✅ Stream opened


[mjpeg @ 0x149066150] overread 8
[mjpeg @ 0x149066150] overread 8
